# Retrieving Data Act

This code is made for EU laws printed in pdf

In [1]:
import pdfminer
# pip3 install pdfminer.six
from pdfminer.high_level import extract_text
import re
import numpy as np
import pandas as pd
import PyPDF2 as pdf2
from PyPDF2 import PdfFileReader

In [2]:
file = open(r'C:\Users\JVoigt\OneDrive - Universität für Weiterbildung Krems\Dokumente\Python Scripts\champI4.0ns\gesetze_pdf\data_act.pdf', 'rb')
pdf = pdf2.PdfFileReader(file)
totalpages = pdf.numPages                                           #certo

#Ich weiß schon, dass die gesetz am Seite 46 begginnt

law = str()

for p in range(46,totalpages):
        page = pdf.getPage(p)
        
        text = page.extractText()
        law = law + text                                           #importou tudo

# Criando uma lista com todos os artigos separados:
law_art = re.split(r'(Artikel\s\d{1,3}\s\n)', law)          # even position = text / odd position = article number

O texto da lei as vezes contem ```Artikel\s\d{1,3}\s\n``` , o que o meu códio entendeu como se fosse um artigo novo. Eu tentei automatizar a correcao desses erros, mas infelizmente eu nao consegui fazer isso de forma satisfatória. Por questao de tempo, eu vou corrigir na mao:
As posicoes sao 35, 39 e 45

In [3]:
# Corrigindo na mao:

posicoes = [45, 39, 35]

for i in posicoes:
    law_art[i-1] = law_art[i-1] + " " + law_art[i] + " " + law_art[i+1]

#dropando de trás pra frente, pra nao influenciar as outras posicoes
for i in posicoes:
    del law_art[i+1]
    del law_art[i]

In [4]:
gesetzt = 'Data Act'
df_law_art = pd.DataFrame(columns = ['gesetzt', 'artikel', 'absatz', 'text'])

# Selecionando só as posicoes ímpares (que sao os enunciados dos artigos):

artigos = []

for i in range(1,len(law_art)):
    if(i%2!=0):
        artigos.append(i)

#Esse aqui é o loop para fazer o DF final

for i in artigos:
    art_num = re.sub(r'Artikel\s',"",law_art[i])   # pegando o próximo enunciado de artigo
    text = law_art[i+1]                            # acha o texto na próxima posicao
    
    if bool(re.search('\(\d{1,3}\)\s', text)):                 # se achar (num) (Absatz) 
        law_abs = re.split(r'\(\d{1,3}\)\s', text)    #divide os números dos absatz
        
        for abs in range(1,len(law_abs)):
            df_temp = pd.DataFrame({'gesetzt' : [gesetzt],'artikel' : [art_num],'absatz' : [abs],'text': [law_abs[abs]]} )
            df_law_art = pd.concat([df_law_art, df_temp])
    
    else:                                               # Se ele nao acha absatz:
        df_temp = pd.DataFrame({'gesetzt' : [gesetzt],'artikel' : [art_num],'absatz' : ['NA'],'text': [text]} )
        df_law_art = pd.concat([df_law_art, df_temp])

In [6]:
#Salvando o banco com esse artigo:

df_law_art.to_csv("eu_data_act.csv")

Para cada linha do banco, se a coluna Absatz estiver escrito concatenar, pegue o texto e concatene com o texto anterior. Se nao, continue.